# UCR Data

In [1]:
include("generate_artifacts.jl")
data_dir = "../data"
artifact_name = "ucr";

### Retrieve data

In [2]:
using PyCall
using BSON: @save
run = false
if run 
    py"""
    from tslearn.datasets import UCR_UEA_datasets

    X_train, y_train, X_test, y_test = UCR_UEA_datasets().load_dataset('ECG200')
    """
    
    # Relabel: 
    y_train = ifelse.(py"y_train".==-1,0,1)
    y_test = ifelse.(py"y_test".==-1,0,1)


    data = Dict(
        "X_train" => py"X_train",
        "y_train" => y_train,
        "X_test" => py"X_test",
        "y_test" => y_test
    )

    @save joinpath(data_dir, artifact_name * "_data.bson") data
end

In [3]:
using BSON: @load
@load joinpath(data_dir, artifact_name * "_data.bson") data
y_train, X_train, X_test, y_test = values(data)
n_train = size(X_train)[1]
input_dim = size(X_train)[2]
output_dim = 1;

### Classifier

#### MLP

In [4]:
using Flux
# MLP
function mlp(;input_dim, hidden_dim, output_dim, activation=Flux.relu)
    model = Chain(
        Dense(input_dim, hidden_dim, activation),
        Dropout(0.1),
        Dense(hidden_dim, output_dim)
    )  
    return model
end

mlp (generic function with 1 method)

In [5]:
using Flux: DataLoader
kw_args = (input_dim=input_dim,hidden_dim=128,output_dim=output_dim)
model = mlp(;kw_args...)
loss(x, y) = Flux.Losses.logitbinarycrossentropy(model(x), y)
bs = Int(round(n_train/10))
data = DataLoader((X_train[:,:,1]',Flux.unsqueeze(y_train,1)),batchsize=bs)
test_data = zip(Flux.unstack(X_test[:,:,1],1),Flux.unsqueeze(y_test,1));

In [6]:
using Flux.Optimise: update!, ADAM
using Statistics
opt = ADAM()
epochs = 20
avg_loss(data) = mean(map(d -> loss(d[1],d[2]), data))
accuracy(data) = mean(map(d -> mean(round.(Flux.σ.(model(d[1]))) .== d[2]), data))

if false
  for epoch = 1:epochs
    for d in data
      gs = gradient(params(model)) do
        l = loss(d...)
      end
      update!(opt, params(model), gs)
    end
    println("Epoch " * string(epoch))
    @show avg_loss(data)
    @show accuracy(data)
    @show accuracy(test_data)
  end
  @save joinpath(data_dir, artifact_name * "_model.bson") model
end

### Generate artifact

In [7]:
generate_artifact()

┌ Info: Binding ucr_data in Artifacts.toml...
└ @ Main /Users/patrickaltmeyer/Documents/code/AlgorithmicRecourse.jl/dev/artifacts/generate_artifacts.jl:58


MethodError: MethodError: objects of type Bool are not callable